# Crawler

This notebook contains started code structure for creating a crawler on single machine

**Author:** Noshaba Nasir

**Date:** 26/3/2021

**Updated by:** Muhammad Usman Zahoor 17L-4102
    

In [ ]:
import os 
import random
from urllib.parse import urlparse
import requests
import heapq
import urllib.request
from bs4 import BeautifulSoup
import threading
# Add any library to be imported here

In [ ]:
# Crawler Parameters
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
WAITTIME= 15 ; # wait 15 seconds before fetching URLS from 
visited = []

# Add any other global parameters here

# FRONTIER

Frontier should use the Mercator frontier design as discussed in lecture.

Preferably it should be a class and should have the given functions.

*prioritizer* function is a stub right now, it will return a random number  between 1 to f for given URL 

In [ ]:
class frontier:
# add the code for frontier here
# should have functions __init__, get_URL, add_URLs, add_to_backqueue
  def __init__(self):
        '''
        Initializer
        '''
        self.back_queue_time_stamp=[0 for i in range(BACKQUEUES)]
        self.back_queues = [[] for i in range(BACKQUEUES)]
        self.front_queues=[[] for i in range(FRONTQUEUES)]
        self.seed_urls = ["https://docs.oracle.com/en/","https://www.oracle.com/corporate/",
                        "https://en.wikipedia.org/wiki/Machine_learning","https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html",
                        "https://docs.oracle.com/middleware/jet210/jet/index.html","https://en.wikipedia.org/w/api.php","https://en.wikipedia.org/api/",
                        "https://en.wikipedia.org/wiki/Weka_(machine_learning)"]
        for i in self.seed_urls:
          self.addURLs(i)
                      
  def get_URL(self):
    if len(self.back_queues)>0:
      i = BACKQUEUES-1
      while i >=0 :
        if len(self.back_queues[i]) > 0:
          self.back_queue_time_stamp[i]+=WAITTIME 
          return self.back_queues[i].pop(0)
        i-=1
    return self.getURL_from_front()

  #adding to front queue
  def addURLs(self,url):
    priority = prioritizer(url, FRONTQUEUES-1)
    self.front_queues[priority-1].append(url)

    
  def check_empty_backqueue(self):
    for i in range(BACKQUEUES):
      if len(self.back_queues[i])==0:
        return i
    return -1

  def getURL_from_front(self):
    i = FRONTQUEUES-1
    while i >=0 :
      if len(self.front_queues[i]) > 0:
        return self.front_queues[i].pop(0)
      i-=1

  def add_to_backqueue(self):
    flag = True
    while flag == True:
      url = self.getURL_from_front()
      addNew = True
      empty_queue = self.check_empty_backqueue();
      if empty_queue == -1:
        flag = False
      else:
        domain = urlparse(url).netloc
        for i in range(BACKQUEUES):
          if domain == urlparse(self.back_queues[i]).netloc:
            self.back_queues[i].append(url)
            addNew = False

        if addNew == True:
          self.back_queues[empty_queue].append(url)
  # add more functions here



def prioritizer(URL,f):
    """
    Take URL and returns priority from 1 to F
    Right now it like a stub function. 
    It will return a random number from 1 to f for given inputs. 
    """
    return random.randint(1,f)
    

# FILTER URLS

Filter the URLS that are in robots.txt files of server and the have been already processed. 

In [ ]:
# write code for filtering urls here 

def filter_url(url):
  domain = urlparse(url).netloc
  if domain is not None:
    robotPath = "http://"+domain+"/robots.txt"
    robot = requests.get(robotPath)
    my_list = list(robot.text.split('\n'))
    for ur in my_list:
      if (ur.split(':')[0] == "Disallow"):
        nUrl =  "http://" + domain + (ur.split(':')[1]).strip()
        if nUrl == url:
          return False
    return True


## ---------------------Write any other codes/data require to run the crawler above this cell-----------

# Run Crawler

In [ ]:
def get_links(doc,url):
  beauty = BeautifulSoup(doc, 'lxml')
  links_to_return = set()
  links = beauty.findAll('a',href=True)
  for link in links:
    address = link.attrs['href'];
    if 'http' in address:
      links_to_return.add(address)
  return links_to_return
  

def get_data_from_url(url):
  with urllib.request.urlopen(url) as response:
    data = response.read()
    return data

# Theard task
def crawler_thread_task(frontier_,count):
  url = frontier_.get_URL()
  while url is  None:
    url = frontier_.get_URL()
  if url is not None:
    if url[0] != "#": 
      print("Thread "+  str(count)+ " Visiting " + url+"\n")
    links = set()
    try:
      links = get_links(get_data_from_url(url),url)
    except:
      print("Error in "+ "Thread "+ str(count)+"\n")
    visited.append(url)
    for link in links:
      if link not in visited:
        frontier_.addURLs(link);
    



In [ ]:

links = get_links(get_data_from_url("https://en.wikipedia.org/wiki/Machine_learning"),"https://en.wikipedia.org/wiki/Machine_learning")

In [ ]:
# start the threads
frontier_ = frontier()
threads=[]
count = 0
while count<100:
  
  try:
    threads.clear()
    for j in range(THREADS):
      t1 = threading.Thread(target=crawler_thread_task, args=(frontier_,j))
      t1.start()
      threads.append(t1) 
      count+=1
    for j in threads:
      j.join()
  except:
    print("Error")




Thread 0 Visiting https://docs.oracle.com/middleware/jet210/jet/index.html

Thread 1 Visiting https://www.oracle.com/corporate/

Thread 2 Visiting https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html
Thread 3 Visiting https://en.wikipedia.org/w/api.php

Thread 4 Visiting https://en.wikipedia.org/wiki/Weka_(machine_learning)

Thread 5 Visiting https://en.wikipedia.org/wiki/Machine_learning

Thread 6 Visiting https://docs.oracle.com/en/
Thread 7 Visiting https://en.wikipedia.org/api/



Thread 8 Visiting https://foundation.wikimedia.org/wiki/Developer_app_guidelines

Thread 0 Visiting http://ricardo.ecn.wfu.edu/pub/gretl/svm/gretl-svm.pdf

Thread 1 Visiting http://www.causality.inf.ethz.ch/activelearning.php
Thread 2 Visiting http://sds.podval.org


Thread 3 Visiting http://www.csie.ntu.edu.tw/~cjlin/libsvm/COPYRIGHT

Thread 4 Visiting http://www.python.org/download/
Thread 5 Visiting https://github.com/nicolaspanel/node-svm/tree/master


Thread 6 Visiting http://quotenil.com

Thread 7 V

## ------------------------------------------------------End of Notebook---------------------------------------------------